This notebook explores a few different potential models of the Milky Way

In [ ]:
using Printf
using CairoMakie
using Revise
using LilGuys, Arya

In [ ]:
using PythonCall
agama = pyimport("agama")
np = pyimport("numpy")

In [ ]:
include("agama_utils.jl")
include("agama_plots.jl")

In [ ]:
obs_props_filename = ENV["DWARFS_ROOT"] * "/observations/sculptor/observed_properties.toml"
icrs = LilGuys.coord_from_file(obs_props_filename)
icrs_err = LilGuys.coord_err_from_file(obs_props_filename)
gc_scl = LilGuys.transform(Galactocentric, icrs)

##  EP2021

In [ ]:
V_T2GYR

In [ ]:
lmc_coord = ICRS(ra = 80.8,
    dec = -69.8, 
    pmra = 1.910,
    pmdec = 0.229,
    radial_velocity = 262,
    distance = 49.59
    )

lmc_coord_err = ICRS(
    ra=0.3,
    dec=0.3,
    distance=0.54,
    pmra=0.2,
    pmdec=0.47,
    radial_velocity=3,
    )

In [ ]:
gc = LilGuys.transform(Galactocentric, lmc_coord)

In [ ]:
ep20 = load_agama_potential("EP2020.ini")
galpy14 = load_agama_potential("galpy_2014.ini") 

In [ ]:
pot = ep20

In [ ]:
σv = calc_σv_interp(pot)

In [ ]:
lines(log10.(σv.x), σv.y * V2KMS)

In [ ]:
orbit_nofric = calc_orbit(gc, pot, time=-10/T2GYR)

In [ ]:
Mmw = pyconvert(Float64, pot.enclosedMass(300))

In [ ]:
Mlmc = 10.

In [ ]:
f_fric(pos, vel) = a_dyn_friction(pos, vel, r_s=10, σv=σv, ρ=x->pyconvert(Float64, pot.density(x)), M=Mlmc)

In [ ]:
use_noninacc = 1

In [ ]:
f_grav(pos, vel) = (1 + use_noninacc * Mlmc/Mmw) * pyconvert(Vector{Float64}, pot.force(pos)) 

In [ ]:
f_acc(pos, vel) =  f_grav(pos, vel) .+ f_fric(pos, vel)

In [ ]:
orbit_2 = leap_frog(gc, f_acc, dt_max=1)

In [ ]:
fig, ax, _ = plot_r_t(orbit_nofric)
plot_r_t!(ax, orbit_2)

fig

In [ ]:
fig, ax, p = plot_y_z(orbit_nofric)
lines!(orbit_2.position[2, :], orbit_2.position[3, :])

fig


## Adding a galaxy...

In [ ]:
ep20 = load_agama_potential("EP2020.ini")


In [ ]:
pot = load_agama_potential("EP2020.ini")

In [ ]:
orbit = calc_orbit(gc, ep20, units=:code, time=-10)

In [ ]:
plot_r_t(orbit)

In [ ]:
plot_y_z(orbit)

In [ ]:
plot_v_circ(ep20)

In [ ]:
mm11 = load_agama_potential("mcmillan11.ini")
mm17 = load_agama_potential("mcmillan17.ini")
bt08 = load_agama_potential("BT08.ini")
piffl14 = load_agama_potential("piffl_14.ini")
J95 = load_agama_potential("J95.ini")
galpy14 = load_agama_potential("galpy_2014.ini")

In [ ]:
potentials_light = [
    "EP20" => ep20,
    "bt08" => bt08,
    "galpy14" => galpy14,
    ]


In [ ]:
potentials_heavy = [
    "EP20" => ep20,
    "mcmillan 11" => mm11,
    "piffl14" => piffl14,
    "J95" => J95,

]

In [ ]:
potentials = [
    "EP20" => ep20,
    "mcmillan 17" => mm17,

]

In [ ]:
my_potentials = {
    "fiducial": EP2020,
    "light": light_mw,
    "heavy": heavy_mw,
    "flat": me_flat,
    "steep": me_steep
}

In [ ]:
for label, pot in my_potentials.items():
    print(label, o.jr(pot))

In [ ]:
o.jr(galpy14)

In [ ]:
o.jr(galpy14_heavy)

In [ ]:
o.jr(EP2020)

In [ ]:
o.jr(ablimit20a)

In [ ]:
o.jr(mcmillan2017)

In [ ]:
for label, pot in my_potentials.items():
    print(label, o.jz(pot))

In [ ]:
o.jz(galpy14)

In [ ]:
o.jz(galpy14_heavy)

In [ ]:
o.jz(EP2020)

In [ ]:
o.jz(mcmillan2017)

In [ ]:
plot_Vc(EP2020_bulge)
plot_Vc(mcmillan2011_bulge)
plot_Vc(galpy14_bulge)
plot_Vc(pouliasis17_bulge)

In [ ]:
plot_Vc(EP2020_thick + EP2020_thick)
plot_Vc(galpy14_disk)
# plot_Vc(galpy14_disk)

plot_Vc(pouliasis17_thick + pouliasis17_thick)
# plot_Vc(pouliasis17b_thick + pouliasis17b_thick)

In [ ]:
plot_Vc(EP2020_halo)
plot_Vc(mcmillan2011_halo)
plot_Vc(mcmillan2017_halo)
plot_Vc(galpy14_halo)
plot_Vc(2*galpy14_halo)

plot_Vc(pouliasis17_halo)
plot_Vc(pouliasis17b_halo)
plot_Vc(ablimit_b_comp["halo"]) # pretty much same as galpy


In [ ]:
dwarf_galaxy_gc.icrs

In [ ]:
o = Orbit(dwarf_galaxy_gc)

In [ ]:
pot = EP2020
ts = np.linspace(0, -5, 10_000) * u.Gyr
o.integrate(ts, pot)

In [ ]:
fig, ax = plt.subplots()

plt.plot(o.R(ts), o.z(ts))
ax.set_aspect(1)

In [ ]:
fig, ax = plt.subplots()

plt.plot(o.y(ts), o.z(ts))

ax.set_aspect(1)

In [ ]:
plt.plot(ts, o.r(ts))

In [ ]:
pos_gc = []

ts = np.linspace(0, 0.1, 10_000) * u.Gyr
o.integrate(ts, pot)

# galpy uses left-handed frame :/
pos_gc = coord.SkyCoord(
    x=- o.x(ts), y=o.y(ts), z=o.z(ts), 
    v_x = -o.vx(ts), v_y=o.vy(ts), v_z=o.vz(ts),
    frame=gc_frame)

pos_icrs = pos_gc.transform_to("icrs")

In [ ]:
plt.scatter(pos_icrs.ra, pos_icrs.dec, c=ts)
plt.colorbar()

plt.plot(o.ra(ts), o.dec(ts))

plt.scatter(dwarf_galaxy.ra, dwarf_galaxy.dec)


tpm = 3e7*u.yr
plt.plot(sculptor.ra + sculptor.pm_ra_cosdec / np.cos(sculptor.dec) * tpm * np.array([0, 1]),
        sculptor.dec + sculptor.pm_dec * tpm * np.array([0,1]),
         zorder=5, color=arya.COLORS[2]
        )

plt.xlabel("RA")
plt.ylabel("dec")

In [ ]:
sc2 = dwarf_galaxy_gc.transform_to(coord.ICRS)

# Alla Vasiliev+21

I have the potential from the Vasiliev et al. (V+21, 2021) paper "Tango for Three...", stored in agama/potentials/vasiliev+2021.
The components of this potential are:

- Multipole expansion of evolving N-body MW potential
- Moving & Evolving multipole expansion of an LMC potential from their N-body simulation
- The non-inertial acceleration of the MW

My goal with the plots below is to investigate these effects and how each one in turn influences both the orbtits of the LMC and Scl. To 2st order, each effect is important but likely a full N-body simulation is required to capture the full nuance of interacting galaxies.

In [ ]:
using CSV, DataFrames

In [ ]:
vasiliev21_frozen = load_agama_potential("vasiliev+21/potential_nolmc.ini")
vasiliev21 = load_agama_potential("vasiliev+21/potential_evolving.ini")
vasiliev21_lmc = load_agama_potential("vasiliev+21/potential_lmc_only.ini")

In [ ]:
# loads in trajectory of lmc in Vasiliev 2021
lmc_file = ENV["DWARFS_ROOT"] * "/agama/potentials/vasiliev+21/trajlmc.txt"
lmc_traj = CSV.read(lmc_file, DataFrame, delim=" ", header = [:time, :x, :y, :z, :v_x, :v_y, :v_z])

filt = lmc_traj.time .<= 15
lmc_traj = lmc_traj[filt, :]
lmc_traj = DataFrame(reverse(eachrow(lmc_traj)))


orbit_lmc_act = Orbit(time=lmc_traj.time  * V_T2GYR/ T2GYR, 
    position = [lmc_traj.x lmc_traj.y lmc_traj.z]',
    velocity = [lmc_traj.v_x lmc_traj.v_y lmc_traj.v_z]' * V_V2KMS / V2KMS
    )



In [ ]:
filename = ENV["DWARFS_ROOT"] * "/agama/potentials/vasiliev+21/accel.txt"
acc_v21 = CSV.read(filename, DataFrame, delim=" ", header = [:time, :x, :y, :z])


## Evolution of the potential

In [ ]:

Ms = []
for i in 1:3
    for j in 1:2
        x = zeros(3)
        x[i] = 1
        x *= (-1)^j

        push!(Ms, [enclosedMass(vasiliev21[4], 100, direction=x, time=t) * V_M2MSUN / M2MSUN for t in times])
    end
end

fig = Figure()
ax = Axis(fig[1,1], xlabel="time / Gyr", ylabel="M lmc")

for M in Ms
    lines!(times, M)
end

fig

In [ ]:
orbit_lmc_act.position[:, 1]

In [ ]:
orbit_lmc_act_resamp

In [ ]:
import NaNMath as nm

In [ ]:
Φ_lmc_v21_evolv = vasiliev21[-1]

In [ ]:
log_r = LinRange(-1, 3, 100)
a = [0,1, 0]


fig = Figure()
ax = Axis(fig[1,1])

N = length(orbit_lmc_act.time)
skip = 10
for i in 1:skip:N
    xs = .+ a' .* 10 .^ log_r
    
    t = orbit_lmc_act.time[i] * T2GYR / V_T2GYR
    ρ = vasiliev21_lmc.density(xs, t=t) |> py2vec
    lines!(log_r, nm.log10.(ρ), color=i, colorrange=(1, N))
end

fig

In [ ]:
log_r = LinRange(-1, 3, 100)
a = [1,0, 0]


fig = Figure()
ax = Axis(fig[1,1])

N = length(orbit_lmc_act.time)
skip = 10
for i in 1:skip:N
    x0 = orbit_lmc_act.position[:, i]
    xs = x0' .+ a' .* 10 .^ log_r
    
    t = orbit_lmc_act.time[i] * T2GYR / V_T2GYR
    ρ = Φ_lmc_v21_evolv.density(xs, t=t) |> py2vec
    lines!(log_r, nm.log10.(ρ), color=i, colorrange=(1, N))
end

fig

In [ ]:
function enclosedMass(vasiliev21, r=60; time=0., direction=[1, 1, 1])
    direction = direction / calc_r(direction)
    a = pyconvert(Vector{Float64}, vasiliev21_lmc.force(r * direction, t=time))
    a = calc_r(a)
    return a * r^2
end
    

In [ ]:
lmc_coord = ICRS(ra = 81,
    dec = -69.75, 
    pmra = 1.8,
    pmdec = 0.35,
    radial_velocity = 260,
    distance = 50.
    )

In [ ]:
lmc_gc = LilGuys.transform(LilGuys.Galactocentric, lmc_coord)

In [ ]:
lmc_traj[lmc_traj.time .== 0, :]

In [ ]:
lmc_gc_2 = LilGuys.Galactocentric(0.448402, -40.9952, -27.5557, -64.2291, -214.992, 205.846)


In [ ]:
LilGuys.transform(LilGuys.ICRS, lmc_gc_2)

In [ ]:
LilGuys.position_of(lmc_gc)

In [ ]:
LilGuys.velocity_of(lmc_gc)

## Utilities

In [ ]:
# some simple python utilities
py2f(x) = pyconvert(Float64, x)
py2vec(x) = pyconvert(Vector{Float64}, x)
py2mat(x) = pyconvert(Matrix{Float64}, x)'

In [ ]:
function make_lmc_pot(Mlmc, r_s; kwargs...)
    Φ_lmc = agama.Potential(;
            type="Spheroid", alpha=1, beta=3, gamma=1, 
            scaleradius = r_s,
            mass = Mlmc, 
            outercutoffradius = 10r_s, 
            cutoffStrength=4,
            kwargs...
        )
    return Φ_lmc
end


In [ ]:
function calc_lmc_orbit(pot, gc_coord; 
        Mlmc = t->15,
        reflex_motion = true,
        dynamical_friction = true,
        Λ = nothing,
        σv = nothing,
        vasiliev_units = false,
        time = -10 / T2GYR, 
        r_s = 8.5 * (Mlmc / 10)^0.6,
        timestep = :adaptive,
        kwargs...
    )

    
    if vasiliev_units
        acc_scale = (V_V2KMS/ V2KMS)^2  * (V_R2KPC / R2KPC)^-1
        v_scale = V2KMS / V_V2KMS
        m_scale = M2MSUN / V_M2MSUN
    else
        v_scale = m_scale = acc_scale = 1
    end
    

    if σv === nothing
        calc_σv_interp(pot, log_r=LinRange(2, -2.0, 1000))
    end

    ρ(x) = py2f(pot.density(x))
    
    Φ_lmc = make_lmc_pot(Mlmc(0) * m_scale, r_s)


    f_fric(pos, vel, t) = dynamical_friction * a_dyn_friction(pos, vel, 
        r_s=r_s,
        σv=σv, ρ=ρ, M=Mlmc(t) * m_scale, Λ = sqrt(calc_r(pos) / 0.8r_s )
    )


    f_grav(pos, vel, t) = py2vec(
        pot.force(pos) - reflex_motion * Mlmc(t) / Mlmc(0) * Φ_lmc.force(-pos)
    )

    f_acc(pos, vel, t) = acc_scale * (f_grav(pos, vel*v_scale, t) .+ f_fric(pos, vel *v_scale, t) )

    orbit = leap_frog(gc_coord, f_acc; time=time, timestep=timestep, kwargs...)

    return orbit
end


In [ ]:
function make_lmc_mw_pot_from_orbit(pot, orbit;
        reflex_motion=true,
        Mlmc=t->15,
        r_s=8.5 , 
        vasiliev_units=false,     
    )
    
    time = orbit.time

    if vasiliev_units
        m_scale = M2MSUN / V_M2MSUN
        time *= T2GYR / V_T2GYR
    else
        m_scale = 1
    end
    
    if orbit.time[2] < orbit.time[1]
        position = orbit.position[:, end:-1:1]
        time = reverse(time)
    end

    centre = vcat(time', position)

    centre = PyArray(centre')
    scale = PyArray(vcat(time',  Mlmc.(time)' ./ Mlmc(0), ones(length(time))')')

    pot_lmc = make_lmc_pot(m_scale * Mlmc(0), r_s, center=centre,scale=scale )

    
    if reflex_motion != 0
        N = length(time)
        a_reflex = pot_lmc.force(zeros(length(time), 3), t=time)
        nptime = PyMatrix(reshape(time,(:, 1)))
        mat_reflex = np.hstack([nptime, -a_reflex])        
        pot_reflex = agama.Potential(type="UniformAcceleration", file=mat_reflex)
        
        Φ = agama.Potential(pot, pot_lmc, pot_reflex)
    else
        Φ = agama.Potential(pot, pot_lmc)
    end

    return Φ
end

In [ ]:
function make_lmc_mw_pot(pot, lmc_gc;
        reflex_motion=true,
        Mlmc=t->15,
        r_s=8.5 * (Mlmc / 10)^0.6, 
        vasiliev_units=false, 
        kwargs...
    
    )
    orbit = calc_lmc_orbit(pot, lmc_gc; Mlmc=Mlmc, r_s=r_s, vasiliev_units=vasiliev_units, reflex_motion=reflex_motion, kwargs...)
    Φ = make_lmc_mw_pot_from_orbit(pot, orbit; Mlmc=Mlmc, r_s=r_s, vasiliev_units=vasiliev_units, reflex_motion=reflex_motion)

    return Φ, orbit
end

## Orbits

In [ ]:
Φ_v21_simple = make_lmc_mw_pot_from_orbit(vasiliev21_frozen, orbit_lmc_act,
    Mlmc=15, 
    vasiliev_units=true,
    reflex_motion=true,
)

In [ ]:
Φ_evolving, orbit_lmc = make_lmc_mw_pot(vasiliev21_frozen, lmc_gc,
    Mlmc=15, 
    σv=x->120, 
    vasiliev_units=true, 
    reflex_motion=true, 
    dynamical_friction=true
)

### Reflex acceleration

In [ ]:
times = acc_v21.time

orbit_lmc_act_resamp = resample(orbit_lmc_act, times * V_T2GYR / T2GYR)

In [ ]:
Φ_lmc_stat = make_lmc_pot(15 * M2MSUN / V_M2MSUN, 10.84)

In [ ]:
Φ_lmc_stat.totalMass() * V_M2MSUN / M2MSUN

In [ ]:
vasiliev21_frozen.totalMass() * V_M2MSUN / M2MSUN

In [ ]:
Φ_lmc_evolving = Φ_evolving[1]

In [ ]:

acc_me = -Φ_lmc_evolving.force(zeros(length(times), 3),  t=times) |> py2mat
acc_me_fmw = vasiliev21_frozen.force(orbit_lmc_act_resamp.position', t=times) *  15 / 80.19 |> py2mat

acc_me_act_orbit = Φ_lmc_stat.force(orbit_lmc_act_resamp.position',  t=times) |>py2mat

# the actual acceleration of the model
acc_v21_mat = hcat(acc_v21.x, acc_v21.y, acc_v21.z)'


In [ ]:
fig = LilGuys.Plots.plot_xyz(acc_me, acc_me_fmw, acc_me_act_orbit, acc_v21_mat, 
    labels = ["my orbit", "rescaled mw-lmc force", "V+21 simple", "V+21 actual"],
    units="",
    xlabel="a_x" ,
    ylabel="a_y",
    zlabel="a_z",
)
fig

In [ ]:
vasiliev21

In [ ]:
fig = Figure()
ax = Axis(fig[1,1], xlabel="time / Gyr", ylabel = L"|a_\textrm{MW}|")

lines!(times, calc_r(acc_me), label="my orbit")
lines!(times, calc_r(acc_me_fmw), label="rescaled mw-lmc force")
lines!(times, calc_r(acc_me_act_orbit), label="V+21 simple")
lines!(times, calc_r(acc_v21_mat), label="V+21 actual")

axislegend(position=:lt)
fig

In the plot above, I compare several different methods for calculating the reflex acceleration of the milkyway.
- `my orbit` is the reflex motion calculated by the force at zero due to the LMC on my fiducial LMC orbit
- `rescaled mw-lmc force` uses the V+21 orbit but instead calculates the force of the MW on the LMC and rescales by the mass ratio
- `V+21 simple` uses the same method as `my orbit` except along the LMC orbit of V+21
- `V+21` contains the actual N-body model's acceleration of the MW

COmparing these models, all are similer except the rescaled mw-lmc force underestimates the acceleration. The actual acceleration deviates from both my orbit and the `V+21 simple` models especially past t=0, but for past times, the magnitude of acceleration is fairly close for each model.

### Orbits

In [ ]:
orbit_scl = calc_orbit(gc_scl, Φ_evolving, time=-5, units=:vasiliev, N=10_001)

In [ ]:
orbit_scl_v21_simple = calc_orbit(gc_scl, Φ_v21_simple, time=-5, units=:vasiliev, N=10_001)

In [ ]:
orbit_scl_nolmc = calc_orbit(gc_scl, vasiliev21_frozen, time=-5, units=:vasiliev)

In [ ]:
v21_noref = agama.Potential(vasiliev21[0], vasiliev21[1], vasiliev21[3], vasiliev21[4])

In [ ]:
orbit_scl_act = calc_orbit(gc_scl, vasiliev21, time=-5, units=:vasiliev, N=10001)

In [ ]:
orbit_lmc_act = resample(orbit_lmc_act, orbit_scl.time)

In [ ]:
orbit_scl_lmc  = orbit_scl - orbit_lmc 
orbit_scl_lmc_act = orbit_scl_act - orbit_lmc_act
orbit_scl_lmc_simple = orbit_scl_v21_simple - orbit_lmc_act

In [ ]:
plot_y_z([
    "vasiliev+21" => orbit_scl_lmc_act,
    "vasiliev+21 simple" => orbit_scl_lmc_simple,
        "approx" => orbit_scl_lmc,

        ])

In [ ]:
plot_y_z([
    "lmc me" => orbit_lmc,
    "lmc vasiliev+21" => orbit_lmc_act,
    "scl me" => orbit_scl,
    "scl vasiliev+21" => orbit_scl_act,
    "scl vasiliev+21 simple" => orbit_scl_v21_simple,

        ])

In [ ]:
orbits = [
    "lmc me" => orbit_lmc,
    "lmc vasiliev+21" => orbit_lmc_act,
    "scl me" => orbit_scl,
    "scl vasiliev+21" => orbit_scl_act,
    "scl vasiliev+21 simple" => orbit_scl_v21_simple,

        ]

In [ ]:
plot_xyz(orbits)

In [ ]:
lmc_me = calc_lmc_orbit(vasiliev21_frozen, lmc_gc, 
    vasiliev_units=true, σv=x->120,
    dynamical_friction=true, reflex_motion=true,
    timestep=:adaptive
)

lmc_point = calc_lmc_orbit(vasiliev21_frozen, lmc_gc, 
    vasiliev_units=true, σv=x->120,
    dynamical_friction=false, reflex_motion=false
)

lmc_dyf = calc_lmc_orbit(vasiliev21_frozen, lmc_gc, 
    vasiliev_units=true, σv=x->120,
    dynamical_friction=true, reflex_motion=false
)


lmc_ref = calc_lmc_orbit(vasiliev21_frozen, lmc_gc, 
    vasiliev_units=true, σv=x->120,
    dynamical_friction=false, reflex_motion=true
)

In [ ]:
orbits = [
    "point" => lmc_point,
    "+reflex" => lmc_ref,
    "+dyn fric" => lmc_dyf,
    "reflex & dyn friction" => lmc_me,
    "vasiliev+2021" => orbit_lmc_act,
    ];

In [ ]:
plot_xyz(orbits)

In [ ]:
fig = Figure()
ax = Axis(fig[1, 1])
plot_v_circ!(Φ_lmc_stat, vasiliev_units=true, log=false, log_r=log10.(LinRange(0.1, 50, 100)))

fig

In [ ]:
Φ_lmc_stat.density([30, 0, 0])

In [ ]:
fig = plot_r_t(orbits, legend=false)

lines!(lmc_orbit.time * T2GYR, r_scl_lmc, label="Scl-LMC ")
axislegend()
fig

In [ ]:
log10(LilGuys.kpc_to_arcmin(LilGuys.calc_break_radius(8.0 / V2KMS, 0.1 / T2GYR), 83.2))

In [ ]:
log10(20)

In [ ]:
fig, ax = FigAxis(
    xlabel = "time / Gyr",
    ylabel = "distance from Scl to LMC / kpc",
    limits=(nothing, nothing, 0, nothing),
    )

lines!(lmc_orbit.time * T2GYR, r_scl_lmc)

fig

In [ ]:
lmc_orbit.time[argmin(r_scl_lmc)] * T2GYR

In [ ]:
minimum(r_scl_lmc)

In [ ]:
lmc_orbit.time ./ scl_lmc_orbit.time

In [ ]:
plot_v_circ(ep20)

In [ ]:
plot_v_circ(vasiliev21, vasiliev_units=true)

# Alla Vasiliev 2024

Similar to above but for ...

- Multipole expansion of evolving N-body MW potential
- Moving & Evolving multipole expansion of an LMC potential from their N-body simulation
- The non-inertial acceleration of the MW

My goal with the plots below is to investigate these effects and how each one in turn influences both the orbtits of the LMC and Scl. To 2st order, each effect is important but likely a full N-body simulation is required to capture the full nuance of interacting galaxies.

In [ ]:
using CSV, DataFrames

In [ ]:
function load_lmc_traj(modelname)
    lmc_file = ENV["DWARFS_ROOT"] * "/agama/potentials/vasiliev24/$modelname/trajlmc.txt"
    lmc_traj = CSV.read(lmc_file, DataFrame, delim=" ", ignorerepeated=true, header = [:time, :x, :y, :z, :v_x, :v_y, :v_z])
    lmc_traj = lmc_traj
    lmc_traj = DataFrame(reverse(eachrow(lmc_traj)))
        
    orbit_lmc_act = Orbit(time=lmc_traj.time  * V_T2GYR/ T2GYR, 
        position = [lmc_traj.x lmc_traj.y lmc_traj.z]',
        velocity = [lmc_traj.v_x lmc_traj.v_y lmc_traj.v_z]' * V_V2KMS / V2KMS
        )
    return orbit_lmc_act
end

In [ ]:
lmc_coord = ICRS(ra = 81,
    dec = -69.75, 
    pmra = 1.8,
    pmdec = 0.35,
    radial_velocity = 260,
    distance = 50.
    )

In [ ]:
lmc_gc = LilGuys.Galactocentric( -0.61,  -41.02,  -26.83,  -69.84,  -221.66,  214.12)

In [ ]:
numpy = pyimport("numpy")

### other units

In [ ]:
pot_frozen = agama.Potential( 
    pydict(type="Spheroid", mass=1.2, scaleRadius=0.2, outerCutoffRadius=1.8, gamma=0, beta=1.8),
    pydict(type="Disk", surfaceDensity=0.088431375, scaleRadius=3.0, scaleHeight=-0.25),
    pydict(type="Spheroid", densityNorm=0.97e-3, scaleRadius=16.5, outerCutoffRadius=500, cutoffStrength=4, beta=3, gamma=1),
    )

In [ ]:
lmc_traj = CSV.read("/arc7/home/dboyea/dwarfs/agama/potentials/vasiliev24/scripts/lmc_orbit.csv", DataFrame)

orbit_lmc_agama = Orbit(time=lmc_traj.t * V_T2GYR /T2GYR ,
        position = [lmc_traj.x lmc_traj.y lmc_traj.z]',
        velocity = ([lmc_traj.vx lmc_traj.vy lmc_traj.vz] / V2KMS)' 
        )

In [ ]:
pwd()

In [ ]:
gridr = numpy.logspace(1, 3, 32)

pot_halo = agama.Potential(type="Spheroid", densityNorm=0.97e-3, scaleRadius=16.5, outerCutoffRadius=500, cutoffStrength=4, beta=3, gamma=1)

df = agama.GalaxyModel(pot_halo, agama.DistributionFunction(type="quasispherical", potential=pot_halo))
sigmas = py2mat(df.moments(numpy.column_stack((gridr, gridr*0, gridr*0)), dens=false, vel=false))[1, :] .^ 0.5

sigmafnc = agama.Spline(gridr, sigmas)

In [ ]:
gridr

In [ ]:
df.totalMass()

In [ ]:
df.totalMass()

In [ ]:
sigmas * V2KMS

In [ ]:
Φ_evolving, orbit_lmc = make_lmc_mw_pot(pot_frozen, lmc_gc,
    Mlmc=20, 
    r_s=9,
    σv= x->py2f(sigmafnc(x)), 
    vasiliev_units=false, 
    reflex_motion=true, 
    dynamical_friction=true
)

In [ ]:
extrema(calc_r(orbit_lmc.position))

In [ ]:
plot_r_t([
    "lmc me" => orbit_lmc,
    "agama" => orbit_lmc_agama,

        ])

In [ ]:
plot_y_z([
    "lmc me" => orbit_lmc,
    "lmc vasiliev+21" => orbit_lmc_agama,

        ])

almost exactly matches agama code. Slight differences are likely due to how agama internally grids data and that we use a different unit system than in the notebook. 

## Bound mass

In [ ]:
function load_accel(modelname)
    filename = ENV["DWARFS_ROOT"] * "/agama/potentials/vasiliev24/$modelname/accel.txt"
    df =  CSV.read(filename, DataFrame, delim=" ", ignorerepeated=true, comment="#", header=["time", "x", "y", "z"])
    acc_scale = (V_V2KMS/ V2KMS)^2  * (V_R2KPC / R2KPC)^-1
    df.x .*= acc_scale
    df.y .*= acc_scale
    df.z .*= acc_scale
    df.time * V_T2GYR, [df.x df.y df.z]'
end

In [ ]:
load_accel("L2M10")

In [ ]:
function load_bound_mass(modelname)
    filename = ENV["DWARFS_ROOT"] * "/agama/potentials/vasiliev24/$modelname/boundmass.txt"
    df =  CSV.read(filename, DataFrame, delim=" ", ignorerepeated=true, comment="#", header=["time", "mass"])
    df.mass ./= M2MSUN
    df.time .*= V_T2GYR
    df
end

In [ ]:
boundmass = [
    "L2M10" => load_bound_mass("L2M10"),
    "L2M11" => load_bound_mass("L2M11"),
    "L3M10" => load_bound_mass("L3M10"),
    "L3M11" => load_bound_mass("L3M11"),
    ];

In [ ]:
fig = Figure()
ax = Axis(fig[1,1],
    xlabel = "time / Gyr"
)

for (label, df) in boundmass
    plot!(df.time, df.mass, label = label)
end

axislegend()

fig

### L3M11

In [ ]:
modelname = "L3M11"

In [ ]:
df = load_bound_mass("L3M11")
boundmass_L3M10 = LilGuys.lerp(df.time / T2GYR, df.mass)

In [ ]:
pot_halo = agama.Potential(ENV["DWARFS_ROOT"] * "/agama/potentials/vasiliev24/$modelname/potential_mw_halo.ini")
pot_frozen = agama.Potential(ENV["DWARFS_ROOT"] * "/agama/potentials/vasiliev24/$modelname/potential_mw_init.ini")
pot_evolving = agama.Potential(ENV["DWARFS_ROOT"] * "/agama/potentials/vasiliev24/$modelname/potential.ini")


In [ ]:
orbit_lmc_act = load_lmc_traj(modelname)

In [ ]:
gridr = numpy.logspace(1, 3, 16)

df = agama.GalaxyModel(pot_halo, agama.DistributionFunction(type="quasispherical", potential=pot_halo))
sigmas = py2mat(df.moments(numpy.column_stack((gridr, gridr*0, gridr*0)), dens=false, vel=false))[1, :] .^ 0.5

sigmafnc = agama.Spline(gridr, sigmas)

In [ ]:
sigmas

In [ ]:
Φ_evolving, orbit_lmc = make_lmc_mw_pot(pot_frozen, lmc_gc,
    Mlmc=x->30, 
    r_s=11.7,
    σv= x->py2f(sigmafnc(x)), 
    vasiliev_units=true, 
    reflex_motion=true, 
    dynamical_friction=true,
    dt_max=5,

)

In [ ]:
Φ_evolving_massevolv, orbit_lmc_massevolv = make_lmc_mw_pot(pot_frozen, lmc_gc,
    Mlmc=boundmass_L3M10, 
    r_s=11.7,
    σv= x->py2f(sigmafnc(x)), 
    vasiliev_units=true, 
    reflex_motion=true, 
    dynamical_friction=1.25,
    dt_max=5,

)

In [ ]:
Φ_evolving_tuned, orbit_lmc_tuned  = make_lmc_mw_pot(pot_frozen, lmc_gc,
    Mlmc=x->15, 
    r_s=11.7,
    σv= x->py2f(sigmafnc(x)), 
    vasiliev_units=true, 
    reflex_motion=true, 
    dynamical_friction=1.15,
    dt_max=5,
)

In [ ]:
lmc_traj = CSV.read("/arc7/home/dboyea/dwarfs/agama/potentials/vasiliev24/scripts/L3M11_orbit.csv", DataFrame)

orbit_lmc_agama = Orbit(time=lmc_traj.t * V_T2GYR /T2GYR ,
        position = [lmc_traj.x lmc_traj.y lmc_traj.z]',
        velocity = ([lmc_traj.vx lmc_traj.vy lmc_traj.vz] / V2KMS)' 
        )

In [ ]:
orbits = [
    "lmc me" => orbit_lmc,
    "lmc tuned" => orbit_lmc_tuned,
    "lmc + mass evol" => orbit_lmc_massevolv,
    "vasiliev24" => orbit_lmc_act,
    "agama" => orbit_lmc_agama,

        ]

In [ ]:
LP = LilGuys.MakieExtension

In [ ]:
function get_acc(Φ, time)
    return Φ[-1].force(np.zeros((length(time), 3)), t=time / V_T2GYR) .* (V_V2KMS / V2KMS)^2  |> py2mat
end

In [ ]:
time, acc = load_accel(modelname)

In [ ]:
acc_tuned = get_acc(Φ_evolving_tuned, time)
acc_massevolv = get_acc(Φ_evolving_massevolv, time)
acc_first = get_acc(Φ_evolving, time)


In [ ]:
LP.plot_xyz(acc_first, acc_tuned, acc_massevolv, acc)

In [ ]:
plot_r_t(orbits)

In [ ]:
plot_y_z(orbits)

In [ ]:
orbit_scl_tuned = calc_orbit(gc_scl, Φ_evolving_tuned, time=-5, units=:vasiliev, N=10_001)

In [ ]:
orbit_scl_massevolv = calc_orbit(gc_scl, Φ_evolving_massevolv, time=-5, units=:vasiliev)

In [ ]:
orbit_scl_act = calc_orbit(gc_scl, pot_evolving, time=-5, units=:vasiliev, N=10001)

In [ ]:
orbits = [
    "lmc tuned" => orbit_scl_tuned,
    "lmc + mass evol" => orbit_scl_massevolv,
    "vasiliev24" => orbit_scl_act,
        ]

In [ ]:
plot_r_t(orbits)

In [ ]:
plot_y_z(orbits)

### L2M10

In [ ]:
modelname = "L2M10"

In [ ]:
pot_frozen = agama.Potential(ENV["DWARFS_ROOT"] * "/agama/potentials/vasiliev24/$modelname/potential_mw_init.ini")

In [ ]:
orbit_lmc_act = load_lmc_traj(modelname)

In [ ]:
gridr = numpy.logspace(1, 3, 16)

df = agama.GalaxyModel(pot_frozen, agama.DistributionFunction(type="quasispherical", potential=pot_frozen))
sigmas = py2mat(df.moments(numpy.column_stack((gridr, gridr*0, gridr*0)), dens=false, vel=false))[1, :] .^ 0.5

sigmafnc = agama.Spline(gridr, sigmas)

In [ ]:
df = load_bound_mass(modelname)
boundmass_L2M10 = LilGuys.lerp(df.time, df.mass)

In [ ]:
Φ_evolving, orbit_lmc = make_lmc_mw_pot(pot_frozen, lmc_gc,
    Mlmc=x->20, 
    r_s=9,
    σv= x->py2f(sigmafnc(x)), 
    vasiliev_units=true, 
    reflex_motion=true, 
    dynamical_friction=true,
    dt_max=10
)

In [ ]:
Φ_evolving_massevolv, orbit_lmc_massevolv = make_lmc_mw_pot(pot_frozen, lmc_gc,
    Mlmc=boundmass_L2M10, 
    r_s=9,
    σv= x->py2f(sigmafnc(x)), 
    vasiliev_units=true, 
    reflex_motion=true, 
    dynamical_friction=1.3,
    dt_max=5,

)

In [ ]:
Φ_evolving_tuned, orbit_lmc_tuned  = make_lmc_mw_pot(pot_frozen, lmc_gc,
    Mlmc=x->13, 
    r_s=9,
    σv= x->py2f(sigmafnc(x)), 
    vasiliev_units=true, 
    reflex_motion=true, 
    dynamical_friction=1.2,
    dt_max=5,
)

In [ ]:
orbits = [
    "lmc me" => orbit_lmc,
    "lmc tuned" => orbit_lmc_tuned,
    "lmc + mass evol" => orbit_lmc_massevolv,
    "vasiliev24" => orbit_lmc_act,
        ]

In [ ]:
plot_r_t(orbits)

In [ ]:
plot_y_z(orbits)

In [ ]:
time, acc = load_accel(modelname)

In [ ]:
boundmass_L2M10(0)

In [ ]:
acc_tuned = get_acc(Φ_evolving_tuned, time)
acc_massevolv = get_acc(Φ_evolving_massevolv, time)
acc_first = get_acc(Φ_evolving, time)


In [ ]:
LP.plot_xyz(acc_first, acc_tuned, acc_massevolv, acc, labels = ["me", "tuned", "mass evolv", "act"])

In [ ]:
accs = [
    "me" => acc_first, 
    "tuned" => acc_tuned,
    "mass evolv" => acc_massevolv, 
    "act" => acc
    ]

In [ ]:
fig = Figure()
ax = Axis(fig[1,1], xlabel = "time", ylabel = "MW acceleration")

for (label, a) in accs
    lines!(time * V_T2GYR, calc_r(a), label=label)

end
axislegend(position=:lt)
fig

In [ ]:
orbit_scl_tuned = calc_orbit(gc_scl, Φ_evolving_tuned, time=-10, units=:vasiliev, N=10_001)

In [ ]:
orbit_scl = calc_orbit(gc_scl, Φ_evolving, time=-10, units=:vasiliev)

In [ ]:
orbit_scl_massevolv = calc_orbit(gc_scl, Φ_evolving_massevolv, time=-10, units=:vasiliev)

In [ ]:
orbit_scl_act = calc_orbit(gc_scl, pot_evolving, time=-10, units=:vasiliev, N=10001)

In [ ]:
orbits = [
    "lmc " => orbit_scl,
    "lmc tuned" => orbit_scl_tuned,
    "lmc + mass evol" => orbit_scl_massevolv,
    "vasiliev24" => orbit_scl_act,
        ]

In [ ]:
plot_r_t(orbits)

In [ ]:
plot_y_z(orbits)

### L2M11

In [ ]:
modelname = "L2M11"

In [ ]:
pot_frozen = agama.Potential(ENV["DWARFS_ROOT"] * "/agama/potentials/vasiliev24/$modelname/potential_mw_init.ini")

In [ ]:
orbit_lmc_act = load_lmc_traj(modelname)

In [ ]:
gridr = numpy.logspace(1, 3, 16)

df = agama.GalaxyModel(pot_frozen, agama.DistributionFunction(type="quasispherical", potential=pot_frozen))
sigmas = py2mat(df.moments(numpy.column_stack((gridr, gridr*0, gridr*0)), dens=false, vel=false))[1, :] .^ 0.5

sigmafnc = agama.Spline(gridr, sigmas)

In [ ]:
Φ_evolving, orbit_lmc = make_lmc_mw_pot(pot_frozen, lmc_gc,
    Mlmc=20, 
    r_s=9,
    σv= x->py2f(sigmafnc(x)), 
    vasiliev_units=true, 
    reflex_motion=true, 
    dynamical_friction=true,
)

In [ ]:
plot_r_t([
    "lmc me" => orbit_lmc,
    "vasiliev24" => orbit_lmc_act,

        ])

In [ ]:
plot_y_z([
    "lmc me" => orbit_lmc,
    "vasiliev24" => orbit_lmc_act,

        ])

### L3M10

In [ ]:
modelname = "L3M10"

In [ ]:
pot_frozen = agama.Potential(ENV["DWARFS_ROOT"] * "/agama/potentials/vasiliev24/$modelname/potential_mw_init.ini")

In [ ]:
orbit_lmc_act = load_lmc_traj(modelname)

In [ ]:
gridr = numpy.logspace(1, 3, 16)

df = agama.GalaxyModel(pot_frozen, agama.DistributionFunction(type="quasispherical", potential=pot_frozen))
sigmas = py2mat(df.moments(numpy.column_stack((gridr, gridr*0, gridr*0)), dens=false, vel=false))[1, :] .^ 0.5

sigmafnc = agama.Spline(gridr, sigmas)

In [ ]:
Φ_evolving, orbit_lmc = make_lmc_mw_pot(pot_frozen, lmc_gc,
    Mlmc=30, 
    r_s=11.7,
    σv= x->py2f(sigmafnc(x)), 
    vasiliev_units=true, 
    reflex_motion=true, 
    dynamical_friction=true,
)

In [ ]:
plot_r_t([
    "lmc me" => orbit_lmc,
    "vasiliev24" => orbit_lmc_act,

        ])

In [ ]:
plot_y_z([
    "lmc me" => orbit_lmc,
    "vasiliev24" => orbit_lmc_act,

        ])

# MC with LMC

In [ ]:
include("agama_utils.jl")

In [ ]:
icrs_samples = LilGuys.rand_coords(icrs, icrs_err, 1000);
gc_samples = LilGuys.transform.(Galactocentric, icrs_samples);

In [ ]:
icrs_df = LilGuys.to_frame(icrs_samples)

In [ ]:
ic = make_agama_init(gc_samples)

In [ ]:
lmc_orbit.time

lmc.times

In [ ]:
orbits = calc_orbits(gc_samples, ep20, time=-5 / T2GYR);

In [ ]:
plot_y_z(orbits[1].second)

In [ ]:
plot_r_t(orbits[1:100], alpha=0.2, color=:black, legend=false)

In [ ]:
orbits = calc_orbits(gc_samples, vasiliev21_frozen, time=-5 / V_T2GYR, units=:vasiliev);

In [ ]:
plot_y_z(orbits[1].second)

In [ ]:
plot_r_t(orbits[1:100], alpha=0.2, color=:black, legend=false)

In [ ]:
orbits = calc_orbits(gc_samples, vasiliev21, time=lmc_traj.time, units=:vasiliev);

In [ ]:
plot_r_t(orbits[1:100], alpha=0.2, color=:black, legend=false)

In [ ]:
rs = [calc_r(lmc_orbit.position, o.position) for (l, o) in orbits];

In [ ]:
rs2 = [calc_r(orbit_lmc_me.position, o.position) for (l, o) in orbits];

In [ ]:
scl_lmc_times = lmc_orbit.time[argmin.(rs)] * T2GYR

In [ ]:
scl_lmc_mins = minimum.(rs)

In [ ]:
hist(scl_lmc_times)

In [ ]:
hist(scl_lmc_mins)

In [ ]:
scatter(icrs_df.pmra, scl_lmc_mins)

In [ ]:
scatter(icrs_df.pmdec, scl_lmc_mins)

In [ ]:
scatter(icrs_df.distance, scl_lmc_mins)

In [ ]:
scatter(icrs_df.distance, scl_lmc_times)

In [ ]:
fig, ax = FigAxis(
    xlabel = "time / Gyr",
    ylabel = "distance from Scl to LMC / kpc",
    limits=(nothing, nothing, 0, nothing),
    )


for r in rs
    lines!(lmc_orbit.time * T2GYR, r, alpha=0.05, color=:black)
end

fig

In [ ]:
orbits[1].second.position

In [ ]:
fig, ax = FigAxis(
    xlabel = "time / Gyr",
    ylabel = "distance  / kpc",
    limits=(nothing, nothing, 0, nothing),
    )


lines!(lmc_orbit.time * T2GYR, rs[1], label="Scl - LMC")
lines!(lmc_orbit.time * T2GYR, rs2[1], label="Scl - LMC point particle")

lines!(orbits[1].second.time * T2GYR, calc_r(orbits[1].second.position), label="Scl - MW")

axislegend(position=:lb)

fig